In [ ]:
!pip install --upgrade git+https://github.com/huggingface/diffusers.git
# !pip install diffusers
!pip install --upgrade git+https://github.com/huggingface/transformers/
# !pip install transformers
!pip install accelerate==0.12.0
!pip install scipy
!pip install ftfy
!pip install gradio==3.16.0
!git clone https://github.com/qunash/stable-diffusion-2-gui
%cd stable-diffusion-2-gui

#@markdown ### ⬅️ Run this cell
#@markdown ---
#@markdown ### Go to Menu > Runtime > Change runtime, in spremeni "Notebook settings" na GPU

install_xformers = True 

if install_xformers:
  import os
  from subprocess import getoutput

  os.system("pip install --extra-index-url https://download.pytorch.org/whl/cu113 torch torchvision==0.13.1+cu113")
  os.system("pip install triton==2.0.0.dev20220701")
  gpu_info = getoutput('nvidia-smi')
  if("A10G" in gpu_info):
      os.system(f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl")
  elif("T4" in gpu_info):
      os.system(f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+1515f77.d20221130-cp38-cp38-linux_x86_64.whl")


# ### install xformers
# from IPython.utils import capture
# from subprocess import getoutput
# from re import search

# with capture.capture_output() as cap:
    
#     smi_out = getoutput('nvidia-smi')
#     supported = search('(T4|P100|V100|A100|K80)', smi_out)

#     if not supported:
#       while True:
#         print("\x1b[1;31mThe current GPU is not supported, try starting a new session.\x1b[0m")
#     else:
#       supported = supported.group(0)

# !pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/{supported}/xformers-0.0.13.dev0-py3-none-any.whl
# !pip install -q https://github.com/ShivamShrirao/xformers-wheels/releases/download/4c06c79/xformers-0.0.15.dev0+4c06c79.d20221201-cp38-cp38-linux_x86_64.whl

# Run the app

In [ ]:
#@title ⬇️🖼️
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler, DDIMScheduler #hugging face - https://huggingface.co
import gradio as gr
import torch
from PIL import Image
import random
import gc

state = None
current_steps = 25

# model_id = 'stabilityai/stable-diffusion-2'
model_id = 'sivar/legostyle1-5' # dataset LAION-5B -  5,8 milijarde parov slik in besedil 768x768

scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler") #naloži  scheduler_config.json

# scheduler - samo 20 korakih ustvari visokokakovostne vzorce, precej dobre vzorce pa lahko ustvari tudi v samo 10 korakih.



pipe_i2i = None


if torch.cuda.is_available():
   torch.backends.cudnn.benchmark = True
   torch.backends.cuda.matmul.allow_tf32 = True



def error_str(error, title="Napaka"):
    gc.collect()
    torch.cuda.empty_cache()
    return f"""#### {title}
            {error}"""  if error else ""
    

def update_state(new_state):
  global state
  state = new_state

def update_state_info(old_state):
  if state and state != old_state:
    return gr.update(value=state)


def get_i2i_pipe(scheduler): #grafični cevovod
    
    update_state("Nalaganje slike v model slike...")

    pipe = StableDiffusionImg2ImgPipeline.from_pretrained( #naloži model v ram, katerega - CPU lahko dostopa
      model_id,
     # revision="fp16", #različica modela - repozitorj
      torch_dtype=torch.float16, #tip cevovodnih izračunov
      scheduler=scheduler
    )
    if torch.cuda.is_available():
      pipe.enable_xformers_memory_efficient_attention()
      #pipe.enable_attention_slicing()
    pipe.to("cuda") #premakne se v GPU
    return pipe




def pipe_callback(step: int, timestep: int, latents: torch.FloatTensor):
    update_state(f"{step}/{current_steps} korakov")#\nTime left, sec: {timestep/100:.0f}")

def inference(guidance, steps, width=768, height=768, seed=0, img=None, strength=0.5):

    update_state(" ")

    
    if seed == 0:
        seed = random.randint(0, 2147483647)

    generator = torch.Generator('cuda').manual_seed(seed)
    #prompt = "made only out of Legos" #style LegoBlocksStyle
    prompt = "LegoStyle style, smooth objects, high resolution"
    #neg_prompt = "hair, curve, circle, blurry, drawing, cartoon illustration"
    neg_prompt = "curve, circle, blurry, drawing, cartoon illustration"
    n_images = 1

    try:
        if img is None:
            return None, gr.update(visible=True, value=error_str("Slika je potrebna za način Slika v sliko"))
        return img_to_img(prompt, n_images, neg_prompt, img, strength, guidance, steps, width, height, generator, seed), gr.update(visible=False, value=None)
    except Exception as e:
        return None, gr.update(visible=True, value=error_str(e))



def img_to_img(prompt, n_images, neg_prompt, img, strength, guidance, steps, width, height, generator, seed):

    global pipe_i2i
    if pipe_i2i is None:
      pipe_i2i = get_i2i_pipe(scheduler)


    
    #img = Image.open('C:\\Users\\sivar\\OneDrive\\Pictures\\Hering.jpg')
    ratio = min(height / img.height, width / img.width)
    img = img.resize((int(img.width * ratio), int(img.height * ratio)), Image.LANCZOS) #Image.Resampling.LANCZOS
    result = pipe_i2i(
      prompt,
      num_images_per_prompt = n_images,
      negative_prompt = neg_prompt, #kar nočemo imeti vključenega
      image = img,
      num_inference_steps = int(steps), #  označuje število korakov odstranjevanja šumov. -  Stabilna difuzija se začne z naključnim šumom in ga večkrat odstrani, da se ustvari smiselna slika. 
      strength = strength, #  je vrednost med 0 in 1. Označuje količino transformacije, ki jo je mogoče uporabiti za vhodno sliko.
      guidance_scale = guidance, # v kolikšni meri vhodni besedilni poziv vpliva na ustvarjanje slik. (večja vrednost - nižja kvaliteta)
      generator = generator,
      callback=pipe_callback).images

    update_state(f"Opravljeno. Seme: {seed}")

    #popuca memory
    gc.collect()
    torch.cuda.empty_cache()
        
    return result


def on_steps_change(steps):
  global current_steps
  current_steps = steps


css = """.main-div div{display:inline-flex;align-items:center;gap:.8rem;font-size:1.75rem}.main-div div h1{font-weight:900;margin-bottom:7px}.main-div p{margin-bottom:10px;font-size:94%}a{text-decoration:underline}.tabs{margin-top:0;margin-bottom:0}#gallery{min-height:20rem}
"""
with gr.Blocks(css="style.css") as demo:
    gr.HTML(
        f"""
          <div class="main-div">
            <div>
              <h1>Uporaba orodja Stable Diffusion za preslikavo slike v oblike Lego</h1>
            </div><br>
            <p> Uporabljen model: <a href="https://huggingface.co/sivar/legoblocksstyle" target="_blank">LegoBlocks.ckpt</a></p>
          </div>
        """
    )
    with gr.Row():
        
        with gr.Column(scale=55):
          with gr.Group():
              with gr.Row():
                generate = gr.Button(value="Generiraj").style()

              gallery = gr.Gallery(label="Ustvarjene slike", show_label=False).style(grid=[2], height="auto")
          state_info = gr.Textbox(label="State", show_label=False, max_lines=2).style(container=False)
          error_output = gr.Markdown(visible=False)

        with gr.Column(scale=45):
          
          with gr.Group():
            image = gr.Image(label="Slika", type="pil")
            strength = gr.Slider(label="Moč šuma", minimum=0, maximum=1, step=0.01, value=0.5)
          with gr.Group():
            with gr.Row():
              guidance = gr.Slider(label="Prisotnost teksta", value=7.5, maximum=15)
              steps = gr.Slider(label="Koraki", value=current_steps, minimum=2, maximum=100, step=1)

            with gr.Row():
              width = gr.Slider(label="Širina", value=768, minimum=64, maximum=1024, step=8)
              height = gr.Slider(label="Višina", value=768, minimum=64, maximum=1024, step=8)

            seed = gr.Slider(0, 2147483647, label='Seme (0 = naključno)', value=0, step=1)


    steps.change(on_steps_change, inputs=[steps], outputs=[], queue=False)
    
    inputs = [guidance, steps, width, height, seed, image, strength] #vhodne vrednosti 
    outputs = [gallery, error_output] #izhodne vrednosti

    generate.click(inference, inputs=inputs, outputs=outputs)

    demo.load(update_state_info, inputs=state_info, outputs=state_info, every=0.5, show_progress=False) #event

    

demo.queue() #čakalna vrsta
demo.launch(debug=True, share=True, height=768) #zagon